In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from datetime import datetime
import numpy as np

In [2]:
df1 = pd.DataFrame(pd.read_csv('combined.csv'))
df2 = pd.DataFrame(pd.read_csv('metro_kpi.csv'))
df = pd.merge(df1, df2, on='Date', how='outer')
#print(df.isnull().sum())
#df['Date'] = pd.to_datetime(df['Date'])
df.dtypes

FileNotFoundError: File b'combined.csv' does not exist

In [ ]:
# Change Elevator Availability from string to float
df['ElevatorAvail'] = df['ElevatorAvail'].str.rstrip('%').astype('float')
df['ElevatorAvail'] = df['ElevatorAvail'] /100
df.head()

In [ ]:
# Add separate columns for years and months
df['Year'] = ''
df['Month'] = ''
#df['ChDate'] = ''

In [ ]:
# Add Year and month values for the future comparison
for ind, row in df.iterrows():
    df.at[ind, 'Year'] = row['Date'][:-6]
    df.at[ind, 'Month'] = row['Date'][5:-3]
    #df.at[ind, 'ChDate'] = row['Year'] + row['Month']
df.head()

In [ ]:
df['Year'] = df['Year'].astype('int')
df['Month'] = df['Month'].astype('int')

In [ ]:
df.head()

In [ ]:
# Create a list of df columns name
features = [column for column in df]

In [ ]:
# Visualize a relationship between months of a year and the features
plt.figure(figsize=(15,60))
x = 1
for feature in features:
    if (feature != 'Date') & (feature != 'Year') & (feature != 'Month') & (feature != 'ChDate'):
        plt.subplot(11,1,x)
        sns.pointplot(x=df['Month'], y=df[feature], hue=df['Year'])
        plt.title(f'Changes in {feature} over the years')
        plt.legend(bbox_to_anchor=(1.02, 1), loc=0, borderaxespad=0.)
        plt.yticks(rotation=65)
        plt.grid(True)
        x += 1

In [ ]:
# Visualize a correlation between Metro ridership and KPIs
plt.figure(figsize=(15,35))
x = 1
for feature in features:
    if (feature != 'Date') & (feature != 'Bus') & (feature != 'Ridership') & (feature != 'Taxi') & (feature != 'Uber') & (feature != 'Year') & (feature != 'Month'):
        plt.subplot(8,2,x)
        sns.regplot(data=df, x=df[feature], y=df['Ridership'])
        plt.title(f'Correlation between Metro ridership and {feature}')
        x += 1
plt.tight_layout()

In [ ]:
# Create a mask for all NaN values
masks = []
masks.extend((~np.isnan(df['Bus']), ~np.isnan(df['Ridership']), ~np.isnan(df['Taxi']), ~np.isnan(df['Uber']),
             ~np.isnan(df['ROTP']), ~np.isnan(df['RailReliability']), ~np.isnan(df['MetroAccessOTP']),
             ~np.isnan(df['EscalatorAvail']), ~np.isnan(df['ElevatorAvail']), ~np.isnan(df['TotalInjuries']),
             ~np.isnan(df['Crime'])))
ttt = masks[4] & masks[1]

In [ ]:
# Collect Ridership/KPI coefficients and put them in a dictionary
reg_data = {}
plt.figure(figsize=(15,33))
c = 1
for x in range(len(features)):
    if (features[x] != 'Date') & (features[x] != 'Bus') & (features[x] != 'Ridership') & (features[x] != 'Taxi') & (features[x] != 'Uber') & (features[x] != 'Year') & (features[x] != 'Month'):
        
        reg_data[features[x]] = stats.linregress(df[features[x]][masks[x-1] & masks[1]],
                                                 df['Ridership'][masks[x-1] & masks[1]])
        fit = (reg_data[features[x]][0] * df[features[x]]) + reg_data[features[x]][1]
        
        # Check if coefficients are correct
        plt.subplot(8,2,c)
        plt.scatter(df[features[x]], df['Ridership'], label=(f'{features[x]}'), c='g')
        plt.plot(df[features[x]], fit, 'r--', label='regression line')
        plt.title(f'Linear regression between {features[x]} and Metro Ridership')
        plt.legend()
        c +=1
        

In [ ]:
# Create DF to keep all regression coefficients
coef_df = pd.DataFrame(reg_data).transpose()
coef_df.columns = ['Slope', 'Intercept', 'R', 'P', 'StdError']
coef_df['Coef_determ'] = coef_df['R']**2
#coef_df.to_csv('linear_regression_coeff_KPI.csv')

In [ ]:
transp_data = {}
plt.figure(figsize=(15,10))
c = 1
for x in range(len(features)):
    if (features[x] == 'Bus') | (features[x] == 'Taxi') | (features[x] == 'Uber'):
        
        transp_data[features[x]] = stats.linregress(df[features[x]][masks[x-1] & masks[1]],
                                                 df['Ridership'][masks[x-1] & masks[1]])
        fit = (transp_data[features[x]][0] * df[features[x]]) + transp_data[features[x]][1]
        # Plot correlation
        plt.subplot(2,2,c)
        plt.scatter(df[features[x]], df['Ridership'], label=(f'{features[x]}'), c='b')
        plt.plot(df[features[x]], fit, 'r--', label='regression line')
        plt.title(f'Linear regression between {features[x]} and Metro Ridership')
        plt.legend()
        c +=1


In [ ]:
coef_transp_df = pd.DataFrame(transp_data).transpose()
coef_transp_df.columns = ['Slope', 'Intercept', 'R', 'P', 'StdError']
coef_transp_df['Coef_determ'] = coef_transp_df['R']**2
#coef_df.to_csv('linear_regression_coeff_KPI.csv')
coef_transp_df

In [ ]:
import statsmodels.formula.api as smf
model = smf.ols(formula='Ridership ~ ROTP + RailReliability', data=df).fit()
model.params

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
x_surf, y_surf = np.meshgrid(np.linspace(df.ROTP.min(), df.ROTP.max(), 100),np.linspace(df.RailReliability.min(),
                                                                                        df.RailReliability.max(), 100))
onlyX = pd.DataFrame({'ROTP': x_surf.ravel(), 'RailReliability': y_surf.ravel()})
fittedY=model.predict(exog=onlyX)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(df['ROTP'],df['RailReliability'],df['Ridership'],c='blue', marker='o', alpha=0.5)
ax.plot_surface(x_surf,y_surf,fittedY.reshape(x_surf.shape), color='None', alpha=0.01)
ax.set_xlabel('ROTP')
ax.set_ylabel('RailReliability')
ax.set_zlabel('Ridership')


In [ ]:
X_new= [[0.90,40],[0.85,55],[0.70,40],[0.75,60]]
df_new= pd.DataFrame(X_new,columns=['ROTP','RailReliability'])
predict = model.predict(df_new)
predict